#imports and etc.

In [ ]:
%matplotlib inline
!pip install fermat
!pip install POT

In [ ]:
import ot
import numpy as np
import matplotlib.pylab as pl
import ot.plot
import fermat as F
from scipy.spatial import  distance_matrix


r = np.random.RandomState(42)


def im2mat(img):
    """Converts an image to matrix (one pixel per line)"""
    return img.reshape((img.shape[0] * img.shape[1], img.shape[2]))


def mat2im(X, shape):
    """Converts back a matrix to an image"""
    return X.reshape(shape)


def minmax(img):
    return np.clip(img, 0, 1)

# Define color transfer with fermat and euclidean


In [ ]:
from sklearn.neighbors import NearestNeighbors

def fermat_color_transfer(I1,I2,n=2000,k=30,alpha=2):
  

  Xss = im2mat(I1)
  Xtt = im2mat(I2)
  idx2 = r.randint(Xss.shape[0], size=(n,))
  idx3 = r.randint(Xtt.shape[0], size=(n,))


  Xs = Xss[idx2, :]
  Xt = Xtt[idx3, :]


  n=Xs.shape[0]
  a, b = np.ones((n,)) / n, np.ones((n,)) / n  # uniform distribution on samples

  method='D'
  model=F.Fermat(alpha, path_method=method,k=k)


  data=np.concatenate((Xs,Xt),axis=0)
  #data=data[:,[0,2]]
  base_dist=distance_matrix(data,data)
  model.fit(base_dist)
  fermat_distances = model.get_distances()

  Mfer2 = fermat_distances[0:n,n:2*n]
  Mfer2=np.ascontiguousarray(Mfer2)

  Mfer=fermat_distances[0:n,n:2*n]
  Mfer=np.ascontiguousarray(Mfer)
  Mfer /=Mfer.max()


  Mfer2=Mfer2**2
  Mfer2 /= Mfer2.max()


  Meucl=distance_matrix(data[0:n,:],data[n:2*n,:])
  Meucl /= Meucl.max()


  Meucl2=distance_matrix(data[0:n,:],data[n:2*n,:])**2
  Meucl2 /= Meucl2.max()

  
  pl.figure(4, figsize=(12, 8))
  pl.subplot(2, 3, 4)
  pl.scatter(data[:, 0], data[:, 2],c=data)
  pl.axis([0, 1, 0, 1])
  pl.xlabel('Red')
  pl.ylabel('Blue')
  pl.title('Image 2')
  pl.tight_layout()
  pl.show()


  for M in [Meucl2,Mfer2]:

    G0 = ot.emd(a, b, M, numItermax=1000000)
    transp_Xs_emd=Xt[np.argmax(G0,axis=1),:]

    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(Xs)
    trash,vecinos_idx=neigh.kneighbors(Xss)


    transp_Xss_emd=np.zeros(Xss.shape)
    for i in range (Xss.shape[0]):  
      transp_Xss_emd[i,:] = transp_Xs_emd[vecinos_idx[i],:] +  Xss[i,:]-Xs[vecinos_idx[i],:] 

    transp_Xss_emd=np.array(transp_Xss_emd)

    I1t = minmax(mat2im(transp_Xss_emd, I1.shape))

    pl.figure(4, figsize=(12, 8))

    pl.subplot(2, 3, 1)
    pl.imshow(I1)
    pl.axis('off')
    pl.title('Image 1: Source')
    
    pl.show()
    
    pl.figure(4, figsize=(12, 8))
    pl.subplot(2, 3, 2)
    pl.imshow(I2)
    pl.axis('off')
    pl.title('Image 2: target')
    
    pl.show()
    
    pl.figure(4, figsize=(12, 8))
    pl.subplot(2, 3, 3)
    pl.imshow(I1t)
    pl.axis('off')
    pl.title('Image 1 Adapt')
    
    pl.show()



# Upload data



Select and upload all images from 'data' folder

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving source.jpg to source.jpg
Saving surface1.jpeg to surface1.jpeg
Saving surface2.jpeg to surface2.jpeg
Saving target.jpg to target.jpg
Saving target2.jpeg to target2.jpeg


In [ ]:
# Loading images
import io

I1 = pl.imread('source.jpg').astype(np.float64) / 256
I2 = pl.imread('target2.jpeg').astype(np.float64) / 256


I3 = pl.imread('surface1.jpeg').astype(np.float64) / 256
I4=pl.imread('target.jpg').astype(np.float64) / 256

I2=I2[0:90,50:140,:]
I1=I1[10:100,40:130,:]


Xs = im2mat(I1)
Xt = im2mat(I2)

X3=im2mat(I3)
X4=im2mat(I4)

nb = 300
idx1 = r.randint(X3.shape[0], size=(nb,))
idx2 = r.randint(X4.shape[0], size=(nb,))

X3 = X3[idx1, :]
X4=X4[idx2,:]


# Transfer!

Transfer from one image to another for all possible combinations of the 5 images uploaded. First row is with squared euclidean cost. Second row with squared Fermat distance with $\alpha=2$. 

In [ ]:
I1 = pl.imread('source.jpg').astype(np.float64) / 256
I2 = pl.imread('target.jpg').astype(np.float64) / 256
I3 = pl.imread('surface2.jpeg').astype(np.float64) / 256
I4 = pl.imread('surface1.jpeg').astype(np.float64) / 256
I5 = pl.imread('target2.jpeg').astype(np.float64) / 256

imagenes=[I1,I2,I3,I4,I5]

for i in range(len(imagenes)):
  for j in range(len(imagenes)):
    if i!=j:
      fermat_color_transfer(imagenes[i],imagenes[j])